converte Folhas de pagamento recebidas em formato html para planilhas excell
registrando cada evento como uma linha

# Configuração do ambiente

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import html
import os
import re
from tqdm import tqdm

# Pré Processamento

In [ ]:
def clean_html(arq_html):
  # Salva apenas as linhas que tem 'class'
  # e aplica a formatação para adicionar acentos
  arq_html = [html.unescape(row) for row in arq_html if 'class' in row or 'CLASS' in row]
  # Remove as divs
  for i, row in enumerate(arq_html):
    pattern = r'<[^>]*>'
    new_row = re.sub(pattern, '', row)
    new_row = new_row.replace('\xa0', ' ')
    arq_html[i] = new_row

  return arq_html

In [ ]:
# Esté método funciona para folhas de pagamento das empresas dos grupos:
# CERUS, FIBRA, GA, GESTART, HISEG, PATRIMONIAIS, PRIMEE, UNICA e VIGON
def get_registros_from_html(path, nome_arq='FolhaPagamento'):

  # Abre o arquivo html
  with open(path, 'r', encoding='latin-1') as file:
    arq_html = file.read()
  arq_html = arq_html.split('\n')

  # Remove informações desnecessárias do arquivo
  arq_html = clean_html(arq_html)

  # Declaração de variáveis
  columns = ['Competência','Empregador','Código_Empregado','Nome_Empregado','Cargo','Lotação','Código_Evento','Desc_Evento','Provento','Desconto']
  registros = pd.DataFrame(columns=columns)
  competencia = str()
  empregador = str()
  codigo_emp = str()
  nome_emp = str()
  lotacao = str()
  cargo = str()

  # Total de empregados que serão coletados:
  total_func = 0
  for row in arq_html:
    if row == 'Data e Assinatura':
      total_func += 1

  # A variável start define o começo dos dados de cada empregado
  start = 0

  for _ in range(total_func):

    # A coleta destas informações não é feita por uso de um 'for'
    # pois a depender da empresa, alguma informação pode mudar
    # de posição

    # Encontra a competência
    for i, row in enumerate(arq_html[start:], start=start):
      if row == 'Competência':
        # A competência está uma linha após esta palavra
        competencia = arq_html[i+1]
        break
    # Encontra o empregador
    for i, row in enumerate(arq_html[start:], start=start):
      if row ==  'Empregador':
        # O empregador está uma linha após a palavra
        empregador = arq_html[i+1]
        break
    # Encontra a lotação
    for i, row in enumerate(arq_html[start:], start=start):
      if row == 'Lotação':
        # A lotação está uma linha após a palavra
        lotacao = arq_html[i+1]
        break
    # Encontra o cargo
    for i, row in enumerate(arq_html[start:], start=start):
      if row == 'Cargo':
        # O cargo está uma linha após a palavra
        cargo = arq_html[i+1]
        break
    # Encontra o código e nome do empregado
    for i, row in enumerate(arq_html[start:], start=start):
      if row == 'Empregado':
        # O código e nome estão uma linha após a palavra
        cod_e_nome = arq_html[i+1].split()
        codigo_emp = cod_e_nome[0]
        nome_emp = ' '.join(cod_e_nome[1:])
        break
    # Guarda os eventos do funcionário
    index = 0
    eventos = list()
    registro = list()

    for i, row in enumerate(arq_html[start:], start=start):
      # Os proventos começam após a linha com 'Cod.'
      if row == 'Cod.':
        index = i+1
        break


    while arq_html[index] != 'Total de Proventos':
      registro.append(arq_html[index])
      index += 1

      if len(registro) == 5:
        # Verifica se o código é não nulo
        if arq_html[index-1]:
          eventos.append(registro)
          registro = list()
        # Caso seja nulo, os eventos acabaram
        else:
          break

    # Converte a lista de eventos para DataFrame
    eventos = pd.DataFrame(eventos)
    # Pode acontecer de aparecer um funcionário sem registros
    # caso esteja afastado, então será ignorado
    if not eventos.empty:
      # Larga a coluna com as referências
      eventos = eventos.drop(2, axis='columns')
      # Inverte a ordem as colunas
      eventos = eventos[eventos.columns[::-1]]
      # Atualiza o nome das colunas
      eventos.columns = ['Código', 'Descrição', 'Proventos', 'Descontos']


    for i in range(eventos.shape[0]):
      # Cria uma lista com todos os dados do registro
      registro = [competencia, empregador, codigo_emp, nome_emp, cargo, lotacao]
      registro.extend([eventos['Código'].iloc[i], eventos['Descrição'].iloc[i], eventos['Proventos'].iloc[i], eventos['Descontos'].iloc[i]])
      # Adiciona uma nova linha na tabela de registros
      index = len(registros)
      registros.loc[index] = registro

    # Encontra os dados do próximo funcionário
    for i, row in enumerate(arq_html[start+1:], start=start+1):
      if row == 'Data e Assinatura':
        start = i
        break

  # Salva a tabela como xlsx
  nome_arq = f'{nome_arq}.xlsx'
  registros.to_excel(nome_arq, index=False)

  #return registros


In [ ]:
# Atualiza alguns códigos para evitar conflitos
# Nem sempre é necessário
def update_cods(df):
  # Relação de códigos antigos e novos
  novo = {
      '350': '99A',
      '388': '99B'
  }

  for i in range(df.shape[0]):
    # É convertido para string pois alguns vem como int
    codigo = str(df['Código_Evento'].iloc[i])
    if codigo in novo:
      df.loc[i, 'Código_Evento'] = novo[codigo]

  return df

In [ ]:
# Este método serve para as folhas do grupo CONDONAL
def get_registros_from_html_condonal(path, nome_arq='FolhaPagamento'):

  # Abre o arquivo html
  with open(path, 'r', encoding='latin-1') as file:
    arq_html = file.read()
  arq_html = arq_html.split('\n')

  # Remove informações desnecessárias do arquivo
  arq_html = clean_html(arq_html)


  # Declaração de variáveis
  columns = ['Competência','Empregador','Código_Empregado','Nome_Empregado','Cargo','Lotação','Código_Evento','Desc_Evento','Provento','Desconto']
  registros = pd.DataFrame(columns=columns)
  competencia = str()
  empregador = str()
  codigo_emp = str()
  nome_emp = str()
  lotacao = str()
  cargo = str()

  # Total de empregados que serão coletados:
  total_func = 0
  for row in arq_html:
    if row == 'EMPRESA':
      total_func += 1

  # A variável start define o começo dos dados de cada empregado
  start = 0

  for _ in range(total_func):

    # Encontra o empregador e a lotação
    for i, row in enumerate(arq_html[start:], start=start):
      if row == 'EMPRESA':
        empregador = arq_html[i+6]
        lotacao = arq_html[i+8]
        break

    # Encontra a competência
    for i, row in enumerate(arq_html[start:], start=start):
      if row == 'SEQUÊNCIA':
        codigo_emp = arq_html[i+1]
        nome_emp = arq_html[i+2]
        cargo = arq_html[i+4]
        competencia = arq_html[i+17]+arq_html[i+18]+arq_html[i+19]
        competencia = get_date(competencia)
        break


    # Guarda os eventos do funcionário
    index = 0
    eventos = list()
    registro = list()

    for i, row in enumerate(arq_html[start:], start=start):
      # Os proventos começam após a linha com 'BATIDAS'
      if row == 'BATIDAS':
        index = i+1
        break

    # Não há nenhuma palavra chave que indique o fim dos eventos
    # o fim dos eventos é seguido por uma série de linhas em branco
    # então é feita uma verificação das linhas em branco, se chegar
    # em 15, a coleta é encerrada
    count_nuls = 0
    while count_nuls < 15:
      registro.append(arq_html[index])
      index += 1
      if len(registro) == 14:
        eventos.append(registro)
        registro = list()
      # Contagem linhas em branco
      if arq_html[index]:
        count_nuls = 0
      else:
        count_nuls += 1

    # Converte a lista de eventos para DataFrame
    eventos = pd.DataFrame(eventos)
    # Pode acontecer de aparecer um funcionário sem registros
    # caso esteja afastado, então será ignorado
    if not eventos.empty:
      # Guarda apenas as colunas necessárias
      eventos = eventos[[3, 5, 9, 11]]
      # Atualiza o nome das colunas
      eventos.columns = ['Código', 'Descrição', 'Proventos', 'Descontos']

    for i in range(eventos.shape[0]):
      # Cria uma lista com todos os dados do registro
      registro = [competencia, empregador, codigo_emp, nome_emp, cargo, lotacao]
      registro.extend([eventos['Código'].iloc[i], eventos['Descrição'].iloc[i], eventos['Proventos'].iloc[i], eventos['Descontos'].iloc[i]])
      # Adiciona uma nova linha na tabela de registros
      index = len(registros)
      registros.loc[index] = registro

    # Encontra os dados do próximo funcionário
    for i, row in enumerate(arq_html[start+1:], start=start+1):
      if row == 'EMPRESA':
        start = i
        break

  # Salva a tabela como xlsx
  nome_arq = f'{nome_arq}.xlsx'
  registros.to_excel(nome_arq, index=False)

  #return registros

In [ ]:
def get_date(row):
  correspondencia = {
      '01/2023':   'Janeiro de 2023',
      '02/2023': 'Fevereiro de 2023',
      '03/2023':     'Março de 2023',
      '04/2023':     'Abril de 2023',
      '05/2023':      'Maio de 2023',
      '06/2023':     'Junho de 2023',
      '07/2023':     'Julho de 2023',
      '08/2023':    'Agosto de 2023',
      '09/2023':  'Setembro de 2023',
      '10/2023':   'Outubro de 2023',
      '11/2023':  'Novembro de 2023',
      '12/2023':  'Dezembro de 2023'
  }

  return correspondencia[row]

In [177]:
# Este método lê os registros de folhas de pagamento de arquivos excel
# As empresas do grupo FIBRA entregam neste formato
def get_data_from_xlsx(path, competencia, empregador, start, end, indice):
  data = pd.DataFrame(columns=['Competência',	'Empregador',	'Código_Empregado',	'Nome_Empregado',	'Cargo',	'Lotação',	'Código_Evento',	'Desc_Evento',	'Provento',	'Desconto'])

  columns = ['Numero', 'Nome', 'Jorn.', 'Cargo', 'Admissão', 'SF', 'IR']
  nrows = end - start + 2
  arq = pd.read_excel(path, skiprows=start, nrows=nrows).fillna('')
  arq.columns = columns

  nome = str()
  codigo = str()
  cargo = str()
  index = 1
  registro = list()

  lotacao = arq['Numero'].iloc[0]
  # Percorre todos os usuários, o fim dos dados de cada usuários
  # é seguido do código do próximo
  while arq['Numero'].iloc[index].isnumeric():
    eventos = pd.DataFrame(columns=['Código', 'Descrição', 'Provento', 'Desconto'])

    codigo = arq['Numero'].iloc[index]
    nome = arq['Nome'].iloc[index]
    cargo = arq['Cargo'].iloc[index]
    # Encontra o começo dos eventos
    for i in range(index, nrows):
      if arq['Numero'].iloc[i] == 'Eventos':
        index = i+1
        break

    # Este while coleta todos os eventos
    # Os eventos são encerrados pela palavra 'Totais'
    while arq['Numero'].iloc[index] != 'Totais':
      registro = list()
      # Codigo evento
      registro.append(arq['Numero'].iloc[index])
      # Descrição
      registro.append(arq['Nome'].iloc[index])
      # Provento
      registro.append(arq['Admissão'].iloc[index])
      # Desconto
      registro.append(arq['SF'].iloc[index])

      eventos.loc[len(eventos)] = registro
      index += 1
    index += 1

    #Para cada linha de eventos, cria um registro completo
    for i in range(eventos.shape[0]):
      registro = [competencia, empregador, codigo, nome, cargo, lotacao]
      registro.extend([eventos['Código'].iloc[i], eventos['Descrição'].iloc[i], eventos['Provento'].iloc[i], eventos['Desconto'].iloc[i]])

      data.loc[len(data)] = registro
  nome_arq = f'folha_parte_{indice:04}.xlsx'
  data.to_excel(nome_arq, index=False)

  #return data

In [ ]:
# Limpa os arquivos no diretório
def clear(html=True, xlsx=True):
  files = os.listdir()

  if html:
    for file in files:
      if '.html' in file or '.HTML' in file:
        os.remove(file)

  if xlsx:
    for file in files:
      if '.xlsx' in file:
        os.remove(file)

# Leitura do arquivo html

In [ ]:
# Lê todos os arquivos html e salva seus registros em .xlsx
files = os.listdir()
files = [file for file in files if '.html' in file]
for arq in tqdm(files):
  # Remove o '.html' do nome
  nome = arq[:arq.find('.html')]
  get_registros_from_html(arq, nome)

100%|██████████| 1/1 [00:02<00:00,  2.74s/it]


In [ ]:
files = os.listdir()
files = [file for file in files if '.HTML' in file]
for arq in tqdm(files):
  nome = arq[:arq.find('.html')]
  get_registros_from_html_condonal(arq, nome)

100%|██████████| 11/11 [21:52<00:00, 119.32s/it]


In [176]:
# Lê todos os arquivos xlsx e junta seus registros
# Lista os arquivos .xlsx
files = os.listdir()
files = [file for file in files if '.xlsx' in file]

df = pd.DataFrame()

for arq in tqdm(files):
  # Salva os DataFrames temporários
  df_temp = pd.read_excel(arq)
  # Concatena os DataFrames
  df = pd.concat([df, df_temp])

df.to_excel('Folha_GERAL.xlsx', index=False)

100%|██████████| 2/2 [02:08<00:00, 64.11s/it]


# _

In [178]:
clear()